## Fake News Prediction using Logistic Regression -- Binary Classification 

<b> Dataset Description </b>

train.csv: A full training dataset with the following attributes:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks the article as potentially unreliable, 1: unreliable, 0: reliable

In [12]:
# Importing Dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords   # nltk = natural language tool kit
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [16]:
# Stop words in English 
print(stopwords.words("English"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

During stemming we will remove all these stopwords from our dataset.

<b> Data Preprocessing </b>

In [20]:
df = pd.read_csv('train.csv')

In [24]:
df.shape

(20800, 5)

In [22]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [28]:
# Missing values in the dataset
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [30]:
# Replacing the null values with empty string
df = df.fillna('')

As it is a large dataset we can fill missing values with empty strings.

In [33]:
# Merging the author name and news title 
df['content'] = df['author']+' '+df['title'] 

In [37]:
print(df['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [39]:
# Seperating the data and label
X = df.drop(columns = 'label', axis = 1) # axis = 1 because of removing column
Y = df['label']

In [41]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

<b> Stemming </b><br>
The process of reducing a word to it's root word.<br>
example: actor, actress, acting --> act

In [47]:
port_stem = PorterStemmer()

In [49]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content) # removes everything which are not alphabets
    stemmed_content = stemmed_content.lower()          # converting everything to lower letters
    stemmed_content = stemmed_content.split()          # splitting into lists
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # taking each word and reducing it to it's root word and choose only the words which are not stopwords
    stemmed_content = ' '.join(stemmed_content)        # joining all the words
    return stemmed_content

In [51]:
df['content'] = df['content'].apply(stemming)

In [53]:
print(df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [55]:
# Seperating data and label
X = df['content'].values
Y = df['label'].values

In [57]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [59]:
print(Y)

[1 0 1 ... 0 1 1]


In [61]:
Y.shape

(20800,)

<b>Converting the textual data into numerical data</b>

In [64]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [66]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.4154496266472162
  (2, 9620)	0.4935149294364995
  (2, 5968)	0.34746133867282925
  (2, 5389)	0.38665305511826153
  (2, 3103)	0.4609748958322965
  :	:
  (20797, 13122)	0.24825263521976057
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396505
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.17455348025522197
  (20797, 9518)	0.2954

<b>Splitting the dataset into train and test data</b>

In [70]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

### Logistic Regression

In [73]:
model = LogisticRegression()

In [75]:
model.fit(X_train, Y_train)

LogisticRegression()

In [77]:
# Accuracy score on the training data
X_train_pred = model.predict(X_train)
training_data_acc = accuracy_score(X_train_pred, Y_train)

In [79]:
print('Accuracy score of the training data : ', training_data_acc)

Accuracy score of the training data :  0.9863581730769231


In [81]:
# Accuracy score on testing data
X_test_pred = model.predict(X_test)
testing_data_acc = accuracy_score(X_test_pred, Y_test)

In [83]:
print('Accuracy score of testing data : ', testing_data_acc)

Accuracy score of testing data :  0.9790865384615385


### Making a Predicting System

In [90]:
X_new = X_test[5]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
    print("The news is Real.")
else:
    print("The news is Fake.")

[1]
The news is Fake.


In [92]:
print(Y_test[5])

1
